# Recommendation systems: using Singular Value Decomposition (SVD)
This is brief introduction to recommender systems based on collaborative filtering using SVD and its implementation in web page recommendation.
(This is just a code snippet to give you an idea.)

## Data
The web pages of the website of a financial institution are grouped according to their characteristics (e.g. tags) in certain categories, and for each of them and each user a measure is estimated that summarizes the Average Session on Page (average amount of time spent on a given page), the total number of sessions and other SEO metrics that measure the customer engagement.
They are all normalized in [0,10].

- Inv = Website pages on investments, KPI in [0,10]
- Ins = Website pages on insurance products, KPI in [0,10]
- ReE = Website pages on mortgages, loans, residential financial, KPI in [0,10]
- Pay = Website pages on payments, credit cards, debt cards, KPI in [0,10]
- MeM = Website pages with generic information, KPI in [0,10]

## Goal
The website is dynamic: a dynamic website is a website that displays different types of content every time a user views it, and this display changes depending on rules. 
The idea is: use the information at your disposal for creating a more tailored experience on the site. Based on a user’s previous visit, or similar users' behaviors, a dynamic website can offer similar or related recommendations in terms of web pages, contents, etc. This is a job for a recommendation system.

Let's import data and libraries, and let's have a look at our data.

In [65]:
import numpy as np
import pandas as pd

webpages = pd.read_excel('/Users/raffaelezenti/Dropbox (virtualb)/RaffaeleZenti/MyMatlab/Laboratorio_Politecnico_Milano/WebPagesKPI.xlsx') # Only reads col1, col2, col3. col0 will be ignored


In [41]:
print(webpages)

               User ID  Inv  Ins  ReE  Pay  MeM
0     OARQ249814647825    5    1    0    0    0
1     VIZP702882221151    0    0    5    0    2
2     FQEO606412621089    6    0    0    0    0
3     TOPH566154866098    0    0    0    8    0
4     TOPH566154869076    6    2    0    0    0
...                ...  ...  ...  ...  ...  ...
2078  UFDM391092504710    7    0    0    0    0
2079  UFDM391092504710    1    0    0    0    5
2080  UXUK848361229470    0    0    9    0    0
2081  ATQU555050383451    2    0    0    1    0
2082  DDFU541993148635    1    0    1    0    2

[2083 rows x 6 columns]


In [66]:
webpages.head()

,User ID,Inv,Ins,ReE,Pay,MeM
0,OARQ249814647825,5,1,0,0,0
1,VIZP702882221151,0,0,5,0,2
2,FQEO606412621089,6,0,0,0,0
3,TOPH566154866098,0,0,0,8,0
4,TOPH566154869076,6,2,0,0,0


In [67]:
webpages.dtypes

User ID    object
Inv         int64
Ins         int64
ReE         int64
Pay         int64
MeM         int64
dtype: object

## Collaborative Filtering
There are many techniques used for finding and recommending many suitable items (item = web page, in this case): **collaborative filtering through SVD is just one of those techniques**. It's very popular.

The assumption of collaborative filtering is that people who have liked or bought an item (a product, a service, a web page, whatever) in the past will also like or buy the same in future. Thus, this approach builds a model based on the past behaviour of users, finding an association between the users and the items. The model is then used to predict the item in which the user may be interested.

We can use SVD to discover structural, statistical relationship between items, and a recommender system can be build easily from this.

Let's see how.
Just like a number, say 30, can be decomposed as factors 30 = 2x5x3, a matrix can also be expressed as multiplication of some other matrices. But because matrices are arrays of numbers, they have their own rules of multiplication: SVD is a linear algebra technique to break down a matrix into the product of a few smaller matrices - see https://en.wikipedia.org/wiki/Singular_value_decomposition, and https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html.


Briefly, SVD assumes a matrix of dimension users x items (which is called Utility Matrix) is decomposed as follows: matrix = U$\cdot$S$\cdot$V'

Let's see an example with a random matrix:

In [68]:
# create a random matrix (no structure inside)
a=np.random.randn(8,5)
print(a)

[[ 0.84648544  1.26070063 -3.07590512  0.60862793  0.69346675]
 [ 0.31753406 -1.18668584 -0.18521525 -1.13049123 -0.15666543]
 [ 0.43980537  0.42486335 -0.32955343 -1.68930254 -0.57923573]
 [ 0.27424359 -0.45297437  0.11550226  1.08716815 -0.04499344]
 [ 0.06046577  1.77249652 -1.06179003  0.97892306 -0.06248195]
 [ 0.82066937  0.66382157 -1.12547882  0.53721618  1.53539702]
 [-0.72570109 -0.93293822  1.19478233  0.33448716 -0.02663278]
 [ 1.22660858 -0.3577489  -0.43986134  0.27384299 -1.19442424]]


In [79]:
from numpy.linalg import svd

# apply SVD
u, s, vh = svd(a, full_matrices=True)

print(u)

[[-0.73996141  0.14516889 -0.08296529  0.19225129 -0.53530661  0.09932034
   0.04019732  0.29917518]
 [ 0.14530541  0.47466912 -0.09233679  0.44927822 -0.27199737 -0.02020144
   0.09532128 -0.67802784]
 [-0.00928369  0.65872012  0.25873047 -0.26600327  0.25109674  0.57828857
   0.10069027  0.1438633 ]
 [-0.00141431 -0.28381723 -0.46808729  0.14242794  0.06105472  0.68357886
  -0.43307063 -0.14661666]
 [-0.41105412 -0.23312088  0.03503475 -0.58034301 -0.01005631  0.08491843
   0.26191713 -0.60228803]
 [-0.39843155 -0.14184959  0.15645025  0.53771755  0.6600027   0.02119492
   0.2553222  -0.07919547]
 [ 0.31659543 -0.29014789 -0.05240048  0.12902612 -0.25184014  0.35636912
   0.76272047  0.15499375]
 [-0.05752743  0.28372232 -0.81859244 -0.16600948  0.27121737 -0.23102018
   0.27582024  0.12463354]]


In [70]:
print(s)

[4.70081724 2.68896011 1.94148647 1.81017566 0.87533429]


Python returns the output as a vector, for simplicity, but it's actually a diagonal matrix containing the singular values (latent factors).

In [71]:
print(np.diag(s))

[[4.70081724 0.         0.         0.         0.        ]
 [0.         2.68896011 0.         0.         0.        ]
 [0.         0.         1.94148647 0.         0.        ]
 [0.         0.         0.         1.81017566 0.        ]
 [0.         0.         0.         0.         0.87533429]]


In [80]:
print(vh)

[[-0.26311395 -0.50554367  0.75316278 -0.23969724 -0.2246947 ]
 [ 0.339741   -0.1152939  -0.31558468 -0.81568957 -0.32609845]
 [-0.48915148  0.42989122  0.11184415 -0.52301258  0.53839864]
 [ 0.18584038 -0.66347327 -0.18354842 -0.03769579  0.70011112]
 [ 0.73589489  0.32577549  0.53567994 -0.0472506   0.25128355]]


Basically, we are looking for the **latent variables, or latent factors**, that hide under the surface of the phenomenon we are analyzing (in this case, the use of web pages):
- U (u in the code) represents the relationship between users and latent factors
- S (s in the code) describes the strength of each latent factor
- V (vh in the code) describes the similarity between items and latent factors.


In [73]:
utility_matrix = np.array([[5, 10, 30, 7, 0, 1, 0, 2], [0, 1, 5, 1, 15, 3, 0, 20],[4, 12, 20, 17, 1, 1, 0, 5],
                    [14, 3, 33, 4, 0, 1, 0, 2], [1, 1, 1, 1, 9, 5, 1, 22],[5, 15, 18, 4, 1, 1, 0, 1]])
rownames = ['Anna', 'Marco', 'Emma', 'Lisa', 'Luca', 'Eva']
colnames = ['Ostriche', 'Cozze', 'Salmone', 'Vongole', 'Salsiccia', 'Costata','Rostelle', 'Hamburger', ]
mydataframe = pd.DataFrame(utility_matrix, index=rownames, columns=colnames)
print(mydataframe)

       Ostriche  Cozze  Salmone  Vongole  Salsiccia  Costata  Rostelle  \
Anna          5     10       30        7          0        1         0   
Marco         0      1        5        1         15        3         0   
Emma          4     12       20       17          1        1         0   
Lisa         14      3       33        4          0        1         0   
Luca          1      1        1        1          9        5         1   
Eva           5     15       18        4          1        1         0   

       Hamburger  
Anna           2  
Marco         20  
Emma           5  
Lisa           2  
Luca          22  
Eva            1  


In [82]:
# apply SVD
u, s, vh = svd(utility_matrix, full_matrices=True)

users_matrix = pd.DataFrame(np.round(100*u).astype(int), index=rownames)
print(users_matrix)

        0   1   2   3   4   5
Anna  -54 -13   2 -14  39  72
Marco -16  69  10 -12  60 -34
Emma  -45   1 -66  57  -3 -20
Lisa  -57 -15  68  22 -22 -31
Luca  -11  69   3   4 -61  38
Eva   -38  -8 -31 -77 -27 -29


In [75]:
print(np.diag(s))

[[60.02729143  0.          0.          0.          0.          0.        ]
 [ 0.         33.9234053   0.          0.          0.          0.        ]
 [ 0.          0.         15.68516924  0.          0.          0.        ]
 [ 0.          0.          0.          8.70326803  0.          0.        ]
 [ 0.          0.          0.          0.          5.31681903  0.        ]
 [ 0.          0.          0.          0.          0.          3.8583524 ]]


In [76]:
products = pd.DataFrame(np.round(100*vh).astype(int), columns=colnames)

print(products)

   Ostriche  Cozze  Salmone  Vongole  Salsiccia  Costata  Rostelle  Hamburger
0       -24    -31      -86      -26         -7       -5         0        -17
1        -7     -4      -18       -1         49       15         2         84
2        35    -65       31      -60          5        1         0          2
3         9    -65       -2       73        -18       -3         0          9
4       -60    -22       26        6         59      -26       -11        -30
5       -67    -11       25      -17        -56       21        10         28
6         0     -1        1        1         10      -13        98         -5
7         2      6        0       -8        -22      -92        -9         30


Please note that the latent factors we get using SVD are **estimate** of the characteristics of the items: the SVD decreases the dimension of the utility matrix by extracting its latent factors. It maps each user and each item into a r-dimensional latent space. This mapping facilitates a "clear" representation of relationships between users and items. Then we will use cosine similarity to find the closest web pages and make a recommendation - see https://en.wikipedia.org/wiki/Cosine_similarity.

Let's work on our webpage data.

In [77]:
# drop the User ID and create the Utility Matrix
webpages.drop(columns=['User ID'],inplace=True)

matrix = webpages.values
u, s, vh = svd(matrix, full_matrices=False)

# little inspection
print(u.shape)
print(s.shape)
print(vh.shape)

(2083, 5)
(5,)
(5, 5)


NOTE 1: By default, the svd() returns a full SVD, but I used a reduced version so we have smaller matrices and we save memory. 
NOTE 2: If we normalize the scores, by subtracting their average rating, we turn low scores into negative numbers and high scores into positive numbers. This is a strong signal - maybe too strong in this case (it's OK with books and movies, but here? these are webpages...).
HINT: try and see what happen.

**The columns of vh correspond to the web pages**.
We can based on vector space model to find which items are most similar to the one we are looking at, using cosine similarity.
And in this stupid example, I try to find the web page that is best match to to first column (column 0), but - HINT - you can get an ordered list.

So if, say, I pick the web page corresponding to the chosen columns, the suggested NBA would be the top result of the cosine similarity ranking.

In [78]:
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))


highest_similarity = -np.inf
highest_sim_col = -1

for col in range(1,vh.shape[1]):
    similarity = cosine_similarity(vh[:,0], vh[:,col])
    if similarity > highest_similarity:
        highest_similarity = similarity
        highest_sim_col = col
 
print("Column %d is most similar to column 0" % highest_sim_col)


Column 2 is most similar to column 0
